# Project Setup

In [ ]:
# Setup path roots
infoviz_root = '/content/drive/My Drive/Semester 3/InfoViz'
json_root = infoviz_root + '/Samples/json'
input_root = infoviz_root + '/Samples/input'
output_root = infoviz_root + '/Samples/output'

## reconstruction.py

In [ ]:
%cd "$infoviz_root/GAST-Net-3DPoseEstimation"

/content/drive/My Drive/Semester 3/InfoViz/GAST-Net-3DPoseEstimation


In [ ]:
%%writefile reconstruction.py

import torch
import torch.nn as nn
import torch
import numpy as np
import json
import cv2
import os
import argparse
from tqdm import tqdm

from tool.mpii_coco_h36m import coco_h36m, mpii_h36m
from common.skeleton import Skeleton
from common.graph_utils import adj_mx_from_skeleton
from common.camera import normalize_screen_coordinates, camera_to_world
from common.generators import *
from model.gast_net import *
from tool.visualization import render_animation


h36m_skeleton = Skeleton(parents=[-1, 0, 1, 2, 0, 4, 5, 0, 7, 8, 9, 8, 11, 12, 8, 14, 15],
                         joints_left=[6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23],
                         joints_right=[1, 2, 3, 4, 5, 24, 25, 26, 27, 28, 29, 30, 31])
adj = adj_mx_from_skeleton(h36m_skeleton)
joints_left, joints_right = [4, 5, 6, 11, 12, 13], [1, 2, 3, 14, 15, 16]
kps_left, kps_right = [4, 5, 6, 11, 12, 13], [1, 2, 3, 14, 15, 16]
rot = np.array([0.14070565, -0.15007018, -0.7552408, 0.62232804], dtype=np.float32)
keypoints_metadata = {'keypoints_symmetry': (joints_left, joints_right), 'layout_name': 'Human3.6M', 'num_joints': 17}

mpii_metadata = {
    'layout_name': 'mpii',
    'num_joints': 16,
    'keypoints_symmetry': [
        [3, 4, 5, 13, 14, 15],
        [0, 1, 2, 10, 11, 12],
    ]
}

coco_metadata = {
    'layout_name': 'coco',
    'num_joints': 17,
    'keypoints_symmetry': [
        [1, 3, 5, 7, 9, 11, 13, 15],
        [2, 4, 6, 8, 10, 12, 14, 16],
    ]
}

h36m_metadata = {
    'layout_name': 'h36m',
    'num_joints': 17,
    'keypoints_symmetry': [
        [4, 5, 6, 11, 12, 13],
        [1, 2, 3, 14, 15, 16],
    ]
}


def parse_args():
    parser = argparse.ArgumentParser(description='Training script')

    # General arguments
    parser.add_argument('-f', '--frames', type=int, default=27, metavar='NAME',
                        help='The number of receptive fields')
    parser.add_argument('-w', '--weight', type=str, default='27_frame_model.bin', metavar='NAME',
                        help='The name of model weight')
    parser.add_argument('-k', '--keypoints-file', type=str, default='./data/keypoints/baseball.json', metavar='NAME',
                        help='The path of keypoints file')
    parser.add_argument('-vi', '--video-path', type=str, default='./data/video/baseball.mp4', metavar='NAME',
                        help='The path of input video')
    parser.add_argument('-vo', '--viz-output', type=str, default='./output/baseball.mp4', metavar='NAME',
                        help='The path of output video')
    parser.add_argument('-kf', '--kpts-format', type=str, default='coco', metavar='NAME',
                        help='The format of 2D keypoints')

    return parser


def load_json(file_path):
    with open(file_path, 'r') as fr:
        video_info = json.load(fr)

    label = video_info['label']
    label_index = video_info['label_index']

    num_person = 2
    num_joints = 17
    num_frames = video_info['data'][-1]['frame_index']
    keypoints = np.zeros((num_person, num_frames, num_joints, 2), dtype=np.float32)
    scores = np.zeros((num_person, num_frames, num_joints), dtype=np.float32)

    for frame_info in video_info['data']:
        frame_index = frame_info['frame_index']

        for index, skeleton_info in enumerate(frame_info['skeleton']):
            pose = skeleton_info['pose']
            score = skeleton_info['score']
            bbox = skeleton_info['bbox']

            if len(bbox) == 0 or index+1 > num_person:
                continue

            pose = np.asarray(pose, dtype=np.float32)
            score = np.asarray(score, dtype=np.float32)
            score = score.reshape(-1)

            keypoints[index, frame_index-1] = pose
            scores[index, frame_index-1] = score

    return keypoints, scores, label, label_index


def load_json_openpose(kpts_dir):
    print('Keypoints dir', kpts_dir)
    kpts_file_list = sorted([name for name in os.listdir(kpts_dir)])
    print('Keypoints files: ', kpts_file_list)

    num_frames = len(kpts_file_list)
    print('Keypoints files count: ', num_frames)

    # TODO add logic for multi person detection
    # num_person = len(img_info['people'])
    num_person = 1
    print(num_person)

    keypoints = np.zeros((num_person, num_frames, 18, 2), dtype=np.float32)  # (M, T, N, C)
    scores = np.zeros((num_person, num_frames, 18))  # (M, T, N)

    # Empty arrays to return for empty json dir
    # keypoints = np.zeros((1, 1, 18, 2), dtype=np.float32)  # (M, T, N, C)
    # scores = np.zeros((1, 1, 18))  # (M, T, N)

    for file_index, kpts_file in tqdm(enumerate(kpts_file_list)):
        with open(kpts_dir + '/' + kpts_file, 'r') as fr:
            img_info = json.load(fr)
    

        for i, human_info in enumerate(img_info['people']):
            kpts_scores = np.asarray(human_info['pose_keypoints_2d'], dtype=np.float32)
            kpts_scores = kpts_scores.reshape(18, 3)

            keypoints[num_person-1, file_index] = kpts_scores[:, :2]
            scores[num_person-1, file_index] = kpts_scores[:, 2]

    return keypoints, scores


def evaluate(test_generator, model_pos, return_predictions=False):
    
    with torch.no_grad():
        model_pos.eval()

        for _, batch, batch_2d in test_generator.next_epoch():

            inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
            if torch.cuda.is_available():
                inputs_2d = inputs_2d.cuda()

            # Positional model
            predicted_3d_pos = model_pos(inputs_2d)

            # Test-time augmentation (if enabled)
            if test_generator.augment_enabled():
                # Undo flipping and take average with non-flipped version
                predicted_3d_pos[1, :, :, 0] *= -1
                predicted_3d_pos[1, :, joints_left + joints_right] = predicted_3d_pos[1, :, joints_right + joints_left]
                predicted_3d_pos = torch.mean(predicted_3d_pos, dim=0, keepdim=True)

            if return_predictions:
                return predicted_3d_pos.squeeze(0).cpu().numpy()


def reconstruction(args):
    """
    Generate 3D poses from 2D keypoints detected from video, and visualize it
        :param chk_file: The file path of model weight
        :param kps_file: The file path of 2D keypoints
        :param viz_output: The output path of animation
        :param video_path: The input video path
        :param kpts_format: The format of 2D keypoints, like MSCOCO, MPII, H36M, OpenPose. The default format is H36M
    """

    # print('Loading 2D keypoints ...')
    # keypoints, scores, _, _ = load_json(args.keypoints_file)

    print('Loading 2D keypoints from OpenPose output ...')
    keypoints, scores = load_json_openpose(args.keypoints_file)

    print('Keypoints shape: ', keypoints.shape)

    # Loading only one person's keypoints
    if len(keypoints.shape) == 4:
        keypoints = keypoints[0]
    assert len(keypoints.shape) == 3

    # Transform the keypoints format from different dataset (MSCOCO, MPII) to h36m format
    if args.kpts_format == 'coco':
        keypoints, valid_frames = coco_h36m(keypoints)
    elif args.kpts_format == 'mpii':
        keypoints, valid_frames = mpii_h36m(keypoints)
    elif args.kpts_format == 'openpose':
        # Convert 'Openpose' format to MSCOCO
        order_coco = [i for i in range(18) if i != 1]
        keypoints = keypoints[:, order_coco]
        keypoints, valid_frames = coco_h36m(keypoints)
    else:
        valid_frames = np.where(np.sum(keypoints.reshape(-1, 34), axis=1) != 0)[0]
        assert args.kpts_format == 'h36m'

    # Get the width and height of video
    cap = cv2.VideoCapture(args.video_path)
    width = int(round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
    height = int(round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    # normalize keypoints
    input_keypoints = normalize_screen_coordinates(keypoints[..., :2], w=width, h=height)

    if args.frames == 27:
        filter_widths = [3, 3, 3]
        channels = 128
    elif args.frames == 81:
        filter_widths = [3, 3, 3, 3]
        channels = 64
    else:
        filter_widths = [3, 3, 3, 3, 3]
        channels = 32

    model_pos = SpatioTemporalModel(adj, 17, 2, 17, filter_widths=filter_widths, channels=channels, dropout=0.05)

    if torch.cuda.is_available():
        model_pos = model_pos.cuda()

    # load trained model
    print('Loading checkpoint', args.weight)
    chk_file = os.path.join('./checkpoint', args.weight)
    checkpoint = torch.load(chk_file, map_location=lambda storage, loc: storage)
    model_pos.load_state_dict(checkpoint['model_pos'])

    receptive_field = model_pos.receptive_field()
    pad = (receptive_field - 1) // 2  # Padding on each side
    causal_shift = 0

    print('Reconstructing ...')
    gen = UnchunkedGenerator(None, None, [input_keypoints[valid_frames]],
                             pad=pad, causal_shift=causal_shift, augment=True,
                             kps_left=kps_left, kps_right=kps_right, joints_left=joints_left, joints_right=joints_right)
    prediction = evaluate(gen, model_pos, return_predictions=True)
    prediction = camera_to_world(prediction, R=rot, t=0)

    # We don't have the trajectory, but at least we can rebase the height
    prediction[:, :, 2] -= np.min(prediction[:, :, 2])

    prediction_new = np.zeros((*input_keypoints.shape[:-1], 3), dtype=np.float32)
    prediction_new[valid_frames] = prediction

    print('Rendering ...')
    anim_output = {'Reconstruction': prediction_new}
    render_animation(keypoints, keypoints_metadata, anim_output, h36m_skeleton, 25, 3000,
                     np.array(70., dtype=np.float32), args.viz_output, limit=-1, downsample=1, size=5,
                     input_video_path=args.video_path, viewport=(width, height), input_video_skip=0)


if __name__ == '__main__':
    parser = parse_args()
    args = parser.parse_args()

    # chk_file = '.../epoch_60.bin'
    # kps_file = '.../2d_keypoints.npz'
    # video_path = '.../sittingdown.mp4'
    # viz_output = '.../output_animation.mp4'

    reconstruction(args)


Overwriting reconstruction.py


## mpii_coco_h36m.py

In [ ]:
%cd "$infoviz_root/GAST-Net-3DPoseEstimation/tool"

/content/drive/My Drive/Semester 3/InfoViz/GAST-Net-3DPoseEstimation/tool


In [ ]:
%%writefile mpii_coco_h36m.py

import numpy as np


# h36m_coco_order = [9, 11, 14, 12, 15, 13, 16, 4, 1, 5, 2, 6, 3]
# coco_order = [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

h36m_coco_order = [1, 2, 3, 4, 5, 6, 9, 11, 12, 13, 14, 15, 16]
coco_order = [10, 11, 12, 7, 8, 9, 13, 1, 2, 3, 4, 5, 6]
spple_keypoints = [10, 8, 0, 7]


def coco_h36m(keypoints):
    temporal = keypoints.shape[0]
    keypoints_h36m = np.zeros_like(keypoints, dtype=np.float32)
    htps_keypoints = np.zeros((temporal, 4, 2), dtype=np.float32)

    # htps_keypoints: head, thorax, pelvis, spine
    # htps_keypoints[:, 0, 0] = np.mean(keypoints[:, 1:5, 0], axis=1, dtype=np.float32)
    # htps_keypoints[:, 0, 1] = np.sum(keypoints[:, 1:3, 1], axis=1, dtype=np.float32) - keypoints[:, 0, 1]
    
    # htps_keypoints[:, 1, :] = np.mean(keypoints[:, 5:7, :], axis=1, dtype=np.float32)
    # htps_keypoints[:, 1, :] += (keypoints[:, 0, :] - htps_keypoints[:, 1, :]) / 3

    # htps_keypoints[:, 2, :] = np.mean(keypoints[:, 11:13, :], axis=1, dtype=np.float32)

    # htps_keypoints[:, 3, :] = np.mean(keypoints[:, [5, 6, 11, 12], :], axis=1, dtype=np.float32)

    # Head
    htps_keypoints[:, 0, :] = keypoints[:, 0, :]
    
    # Pelvis = LHip + RHip / 2
    htps_keypoints[:, 2, :] = (keypoints[:, 10, :] + keypoints[:, 7, :]) / 2

    # Spine = Neck + Pelvis / 2
    htps_keypoints[:, 3, :] = (keypoints[:, 13, :] + htps_keypoints[:, 2, :]) / 2

    # Thorax = Neck + Spine / 2
    htps_keypoints[:, 1, :] = (keypoints[:, 13, :] + htps_keypoints[:, 3, :]) / 2


    # Assign all keypoints 
    keypoints_h36m[:, spple_keypoints, :] = htps_keypoints
    keypoints_h36m[:, h36m_coco_order, :] = keypoints[:, coco_order, :]

    # keypoints_h36m[:, 9, :] -= (keypoints_h36m[:, 9, :] - np.mean(keypoints[:, 5:7, :], axis=1, dtype=np.float32)) / 4
    # keypoints_h36m[:, 7, 0] += 2*(keypoints_h36m[:, 7, 0] - np.mean(keypoints_h36m[:, [0, 8], 0], axis=1, dtype=np.float32))
    # keypoints_h36m[:, 8, 1] -= (np.mean(keypoints[:, 1:3, 1], axis=1, dtype=np.float32) - keypoints[:, 0, 1])*2/3

    # half body: the joint of ankle and knee equal to hip
    # keypoints_h36m[:, [2, 3]] = keypoints_h36m[:, [1, 1]]
    # keypoints_h36m[:, [5, 6]] = keypoints_h36m[:, [4, 4]]

    valid_frames = np.where(np.sum(keypoints_h36m.reshape(-1, 34), axis=1) != 0)[0]
    return keypoints_h36m, valid_frames


h36m_mpii_order = [3, 2, 1, 4, 5, 6, 0, 8, 9, 10, 16, 15, 14, 11, 12, 13]
mpii_order = [i for i in range(16)]
lr_hip_shouler = [2, 3, 12, 13]


def mpii_h36m(keypoints):
    temporal = keypoints.shape[0]
    keypoints_h36m = np.zeros((temporal, 17, 2), dtype=np.float32)
    keypoints_h36m[:, h36m_mpii_order] = keypoints
    # keypoints_h36m[:, 7] = np.mean(keypoints[:, 6:8], axis=1, dtype=np.float32)
    keypoints_h36m[:, 7] = np.mean(keypoints[:, lr_hip_shouler], axis=1, dtype=np.float32)

    valid_frames = np.where(np.sum(keypoints_h36m.reshape(-1, 34), axis=1) != 0)[0]
    return keypoints_h36m, valid_frames




Overwriting mpii_coco_h36m.py


## visualization.py

In [ ]:
%%writefile visualization.py

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import subprocess as sp


elbow_knee_v1 = [5, 15]
elbow_knee_v2 = [2, 12]
wrist_ankle_v1 = [6, 16]
wrist_ankle_v2 = [3, 13]
hip_shoulder = [1, 4, 11, 14]
spine_head = [7, 9]
thorax_neck = [8, 10]


def color_edge(joint_num):
    if joint_num in elbow_knee_v1:
        color = 'peru'
    elif joint_num in elbow_knee_v2:
        color = 'indianred'
    elif joint_num in wrist_ankle_v1:
        color = 'coral'
    elif joint_num in wrist_ankle_v2:
        color = 'brown'
    elif joint_num in hip_shoulder:
        color = 'tan'
    elif joint_num in spine_head:
        color = 'olive'
    else:
        color = 'purple'
    return color


def get_resolution(filename):
    command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
               '-show_entries', 'stream=width,height', '-of', 'csv=p=0', filename]
    with sp.Popen(command, stdout=sp.PIPE, bufsize=-1) as pipe:
        for line in pipe.stdout:
            w, h = line.decode().strip().split(',')
            return int(w), int(h)


def get_fps(filename):
    command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
               '-show_entries', 'stream=r_frame_rate', '-of', 'csv=p=0', filename]
    with sp.Popen(command, stdout=sp.PIPE, bufsize=-1) as pipe:
        for line in pipe.stdout:
            a, b = line.decode().strip().split('/')
            return int(a) / int(b)


def read_video(filename, skip=0, limit=-1):
    w, h = get_resolution(filename)
    
    command = ['ffmpeg',
            '-i', filename,
            '-f', 'image2pipe',
            '-pix_fmt', 'rgb24',
            '-vsync', '0',
            '-vcodec', 'rawvideo', '-']
    
    i = 0
    with sp.Popen(command, stdout = sp.PIPE, bufsize=-1) as pipe:
        while True:
            data = pipe.stdout.read(w*h*3)
            if not data:
                break
            i += 1
            if i > limit and limit != -1:
                continue
            if i > skip:
                yield np.frombuffer(data, dtype='uint8').reshape((h, w, 3))
            

def downsample_tensor(X, factor):
    length = X.shape[0]//factor * factor
    return np.mean(X[:length].reshape(-1, factor, *X.shape[1:]), axis=1)


def render_animation(keypoints, keypoints_metadata, poses, skeleton, fps, bitrate, azim, output, viewport,
                     limit=-1, downsample=1, size=6, input_video_path=None, input_video_skip=0):
    """
    TODO
    Render an animation. The supported output modes are:
     -- 'interactive': display an interactive figure
                       (also works on notebooks if associated with %matplotlib inline)
     -- 'html': render the animation as HTML5 video. Can be displayed in a notebook using HTML(...).
     -- 'filename.mp4': render and export the animation as an h264 video (requires ffmpeg).
     -- 'filename.gif': render and export the animation a gif file (requires imagemagick).
    """
    plt.ioff()
    fig = plt.figure(figsize=(size*(1 + len(poses)), size))
    ax_in = fig.add_subplot(1, 1 + len(poses), 1)
    ax_in.get_xaxis().set_visible(False)
    ax_in.get_yaxis().set_visible(False)
    ax_in.set_axis_off()
    # ax_in.set_title('Input')

    ax_3d = []
    lines_3d = []
    trajectories = []
    radius = 1.5
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, 1 + len(poses), index+2, projection='3d')
        ax.view_init(elev=15., azim=azim)
        ax.set_xlim3d([-radius/2, radius/2])
        ax.set_zlim3d([0, radius])
        ax.set_ylim3d([-radius/2, radius/2])
        # ax.set_aspect('equal')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        ax.dist = 7.5
        # ax.set_title(title)
        ax_3d.append(ax)
        lines_3d.append([])
        trajectories.append(data[:, 0, [0, 1]])
    poses = list(poses.values())

    # Decode video
    if input_video_path is None:
        # Black background
        all_frames = np.zeros((keypoints.shape[0], viewport[1], viewport[0]), dtype='uint8')
    else:
        # Load video using ffmpeg
        all_frames = []
        for f in read_video(input_video_path, skip=input_video_skip, limit=limit):
            all_frames.append(f)
        effective_length = min(keypoints.shape[0], len(all_frames))
        all_frames = all_frames[:effective_length]
        
        keypoints = keypoints[input_video_skip:] # todo remove
        for idx in range(len(poses)):
            poses[idx] = poses[idx][input_video_skip:]
        
        if fps is None:
            fps = get_fps(input_video_path)
    
    if downsample > 1:
        keypoints = downsample_tensor(keypoints, downsample)
        all_frames = downsample_tensor(np.array(all_frames), downsample).astype('uint8')
        for idx in range(len(poses)):
            poses[idx] = downsample_tensor(poses[idx], downsample)
            trajectories[idx] = downsample_tensor(trajectories[idx], downsample)
        fps /= downsample

    initialized = False
    image = None
    lines = []
    points = None
    
    if limit < 1:
        limit = len(all_frames)
    else:
        limit = min(limit, len(all_frames))

    parents = skeleton.parents()
    def update_video(i):
        nonlocal initialized, image, lines, points

        for n, ax in enumerate(ax_3d):
            ax.set_xlim3d([-radius/2 + trajectories[n][i, 0], radius/2 + trajectories[n][i, 0]])
            ax.set_ylim3d([-radius/2 + trajectories[n][i, 1], radius/2 + trajectories[n][i, 1]])

        # Update 2D poses
        joints_right_2d = keypoints_metadata['keypoints_symmetry'][1]
        colors_2d = np.full(keypoints.shape[1], 'black')
        colors_2d[joints_right_2d] = 'red'
        if not initialized:
            image = ax_in.imshow(all_frames[i], aspect='equal')
            
            for j, j_parent in enumerate(parents):
                if j_parent == -1:
                    continue
                    
                if len(parents) == keypoints.shape[1] and keypoints_metadata['layout_name'] != 'coco':
                    # Draw skeleton only if keypoints match (otherwise we don't have the parents definition)
                    lines.append(ax_in.plot([keypoints[i, j, 0], keypoints[i, j_parent, 0]],
                                            [keypoints[i, j, 1], keypoints[i, j_parent, 1]], color='pink'))

                col = color_edge(j)
                for n, ax in enumerate(ax_3d):
                    pos = poses[n][i]
                    lines_3d[n].append(ax.plot([pos[j, 0], pos[j_parent, 0]],
                                               [pos[j, 1], pos[j_parent, 1]],
                                               [pos[j, 2], pos[j_parent, 2]], linewidth=3, zdir='z', c=col))

            points = ax_in.scatter(*keypoints[i].T, 10, color=colors_2d, edgecolors='white', zorder=10)

            initialized = True
        else:
            image.set_data(all_frames[i])

            for j, j_parent in enumerate(parents):
                if j_parent == -1:
                    continue
                
                if len(parents) == keypoints.shape[1] and keypoints_metadata['layout_name'] != 'coco':
                    lines[j-1][0].set_data([keypoints[i, j, 0], keypoints[i, j_parent, 0]],
                                           [keypoints[i, j, 1], keypoints[i, j_parent, 1]])

                for n, ax in enumerate(ax_3d):
                    pos = poses[n][i]
                    lines_3d[n][j-1][0].set_xdata([pos[j, 0], pos[j_parent, 0]])
                    lines_3d[n][j-1][0].set_ydata([pos[j, 1], pos[j_parent, 1]])
                    lines_3d[n][j-1][0].set_3d_properties([pos[j, 2], pos[j_parent, 2]], zdir='z')

            points.set_offsets(keypoints[i])
        
        print('{}/{}      '.format(i, limit), end='\r')

    fig.tight_layout()
    
    anim = FuncAnimation(fig, update_video, frames=np.arange(0, limit), interval=1000/fps, repeat=False)
    if output.endswith('.mp4'):
        Writer = writers['ffmpeg']
        writer = Writer(fps=fps, metadata={}, bitrate=bitrate)
        anim.save(output, writer=writer)
    elif output.endswith('.gif'):
        anim.save(output, dpi=80, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')
    plt.close()


Overwriting visualization.py


# Run GAST-Net

In [ ]:
%cd "$infoviz_root/GAST-Net-3DPoseEstimation"
!python reconstruction.py -f 27 -w 27_frame_model.bin -k "$json_root/Squats-3" -vi "$input_root/Squats-3.mp4" -vo "$output_root/Squats-3-3D.mp4" -kf openpose


/content/drive/My Drive/Semester 3/InfoViz/GAST-Net-3DPoseEstimation
Loading 2D keypoints from OpenPose output ...
Keypoints dir /content/drive/My Drive/Semester 3/InfoViz/Samples/json/Squats-3
Keypoints files:  ['Squats-3_000000000000_keypoints.json', 'Squats-3_000000000001_keypoints.json', 'Squats-3_000000000002_keypoints.json', 'Squats-3_000000000003_keypoints.json', 'Squats-3_000000000004_keypoints.json', 'Squats-3_000000000005_keypoints.json', 'Squats-3_000000000006_keypoints.json', 'Squats-3_000000000007_keypoints.json', 'Squats-3_000000000008_keypoints.json', 'Squats-3_000000000009_keypoints.json', 'Squats-3_000000000010_keypoints.json', 'Squats-3_000000000011_keypoints.json', 'Squats-3_000000000012_keypoints.json', 'Squats-3_000000000013_keypoints.json', 'Squats-3_000000000014_keypoints.json', 'Squats-3_000000000015_keypoints.json', 'Squats-3_000000000016_keypoints.json', 'Squats-3_000000000017_keypoints.json', 'Squats-3_000000000018_keypoints.json', 'Squats-3_000000000019_keyp

In [ ]:
# Sample command
# DO NOT TOUCH
!python reconstruction.py -f 27 -w 27_frame_model.bin -k ./data/keypoints/baseball.json -vi ./data/video/baseball.mp4 -vo ./output/baseball_reconstruction.mp4 -kf coco

Loading 2D keypoints from OpenPose output ...
Keypoints dir ./data/keypoints/baseball.json
Traceback (most recent call last):
  File "reconstruction.py", line 271, in <module>
    reconstruction(args)
  File "reconstruction.py", line 186, in reconstruction
    keypoints, scores = load_json_openpose(args.keypoints_file)
  File "reconstruction.py", line 114, in load_json_openpose
    kpts_file_list = sorted([name for name in os.listdir(kpts_dir)])
NotADirectoryError: [Errno 20] Not a directory: './data/keypoints/baseball.json'


In [ ]:
import os
kpts_file_list = [name for name in os.listdir(json_root + "/Workout-1")]
kpts_file_list

['Workout-1_000000000125_keypoints.json',
 'Workout-1_000000000126_keypoints.json',
 'Workout-1_000000000127_keypoints.json',
 'Workout-1_000000000128_keypoints.json',
 'Workout-1_000000000129_keypoints.json',
 'Workout-1_000000000130_keypoints.json',
 'Workout-1_000000000131_keypoints.json',
 'Workout-1_000000000132_keypoints.json',
 'Workout-1_000000000133_keypoints.json',
 'Workout-1_000000000134_keypoints.json',
 'Workout-1_000000000135_keypoints.json',
 'Workout-1_000000000136_keypoints.json',
 'Workout-1_000000000137_keypoints.json',
 'Workout-1_000000000138_keypoints.json',
 'Workout-1_000000000139_keypoints.json',
 'Workout-1_000000000140_keypoints.json',
 'Workout-1_000000000141_keypoints.json',
 'Workout-1_000000000142_keypoints.json',
 'Workout-1_000000000143_keypoints.json',
 'Workout-1_000000000144_keypoints.json',
 'Workout-1_000000000145_keypoints.json',
 'Workout-1_000000000146_keypoints.json',
 'Workout-1_000000000147_keypoints.json',
 'Workout-1_000000000148_keypoints